<a href="https://colab.research.google.com/github/sameershinde14/ToxicCommentsChallengeKaggle/blob/master/XGBoost_with_Processed_data_Latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pickle_util import load_pickles
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from matplotlib import pyplot as plt
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from scipy.sparse import vstack
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import PredefinedSplit
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from scipy.sparse import vstack

In [0]:
x_classes = ["comment_text"]
y_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_threat"]

In [0]:
(
    train_x,
    valid_x,
    test_x,
    train_y,
    valid_y,
    test_y
) = load_pickles()

In [0]:
word_vectorizer = TfidfVectorizer(lowercase=False,ngram_range=(1,3),max_features=20000)
word_vectorizer = word_vectorizer.fit(train_x["comment_text"])
train_features = word_vectorizer.transform(train_x["comment_text"])
validation_features = word_vectorizer.transform(valid_x["comment_text"])
test_features = word_vectorizer.transform(test_x["comment_text"])

In [0]:
train_y_list = train_y.values
valid_y_list = valid_y.values

In [10]:
# penalty = ['l1', 'l2']
# max_iter = [8, 10, 20, 40, 100]
# C = np.logspace(0, 4, 10)
params = {
        'min_child_weight': [1],
        #'gamma': [0.5, 1],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
         'max_depth': [3]
        }


# hyperparameters = dict(C=C, penalty=penalty, max_iter=max_iter)
# model = LogisticRegression(solver='liblinear')

model = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

ps = PredefinedSplit(test_fold=[-1]*train_features.shape[0] + [0]*validation_features.shape[0])

val_aucs = []
train_aucs = []
best_params = []

X = vstack((train_features,validation_features))
Y = np.concatenate((train_y_list, valid_y_list))

for i in range(len(y_classes)):
    #classifier = GridSearchCV(model, hyperparameters, scoring='roc_auc', cv=ps, verbose=0)
    classifier = GridSearchCV(estimator=model, param_grid=params, scoring='roc_auc', n_jobs=32, cv=ps, verbose=3 )
    
    classifier = classifier.fit(X, Y[:, i])

    valid_y_hat = classifier.predict_proba(validation_features)[:, 1]
    fpr, tpr, thresholds = roc_curve(valid_y_list[:, i], valid_y_hat)
    val_aucs.append(auc(fpr, tpr))

    train_y_hat = classifier.predict_proba(train_features)[:, 1]
    fpr, tpr, thresholds = roc_curve(train_y_list[:, i], train_y_hat)
    train_aucs.append(auc(fpr, tpr))
    
    best_params.append(classifier.best_params_)

Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 out of   1 | elapsed:  6.6min finished


Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 out of   1 | elapsed:  5.8min finished


Fitting 1 folds for each of 1 candidates, totalling 1 fits


...

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 out of   1 | elapsed:  5.6min finished


Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 out of   1 | elapsed:  6.4min finished


Fitting 1 folds for each of 1 candidates, totalling 1 fits


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 out of   1 | elapsed:  5.9min finished


In [13]:
np.mean(val_aucs)

0.9599960315949314

In [14]:
np.mean(train_aucs)

0.9638763795329138

In [0]:
y_hat = classifier.predict(test_features)

In [0]:
test_y_list = test_y.values

In [0]:
te_aucs = []
for i in range(len(y_classes)):
    test_y_hat = classifier.predict_proba(test_features)[:, 1]
    fpr, tpr, thresholds = roc_curve(test_y_list[:, i], test_y_hat)
    te_aucs.append(auc(fpr, tpr))

In [27]:
te_aucs

[0.8496066877004322,
 0.96013257436624,
 0.8867155115346683,
 0.8942077484848014,
 0.8684608301190802,
 0.961938604064096]

In [28]:
np.mean(te_aucs)

0.9035103260448863